In [ ]:
pip install pandas

In [ ]:
pip install seaborn

In [ ]:
pip install plotly

In [ ]:
pip install nbformat

In [ ]:
pip install scikit-bio

In [ ]:
pip install scikit-learn

# librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# sunburst
import plotly.express as px

# agrupamiento y dendrograma
from skbio.diversity import beta_diversity
from scipy.cluster.hierarchy import dendrogram, linkage

# ordenamiento
from sklearn.manifold import MDS

# boxplots
import seaborn as sns

# funciones

In [ ]:
def glom_other_by_prop_rich_share(df, rank, prop):
    """Aglomerar proporciones bajas (definidas en prop) de categorias 
    de una jerarquia taxonomica dada en 'Otros'"""
    rankdf = pd.DataFrame(df[rank].value_counts())
    rankdf['proportion'] = rankdf['count']/rankdf['count'].sum()
    rank_2keep = list(rankdf.loc[rankdf['proportion']>=prop].index.values)
    
    df_lncol = []
    for i in df.index.values:
        crank = df[rank][i]
        if crank not in rank_2keep:
            df_lncol.append('Otros')
            
        else:
            df_lncol.append(crank)
    df[rank +"_n"] = df_lncol

def glom_other_by_prop(df, rank, prop, specific='', glom_d=''):
    """Aglomerar proporciones bajas (definidas en prop) de categorias 
    de una jerarquia taxonomica dada en 'Otros'"""
    rankdf = pd.DataFrame(df[rank].value_counts())
    rankdf['proportion'] = rankdf['count']/rankdf['count'].sum()
    rank_2keep = list(rankdf.loc[rankdf['proportion']>=prop].index.values)

    ordered_props =  pd.pivot_table(df, values='count', index=['site'],
                           columns=[rank],
                   aggfunc="sum").sum().sort_values(ascending=False)/pd.pivot_table(df,
                        values='count', index=['site'],
                           columns=[rank], aggfunc="sum").sum().sum()
    rank_2keep = list(ordered_props.loc[ordered_props>=prop].index.values)
    
    df_lncol = []
    for i in df.index.values:
        crank = df[rank][i]
        if crank not in rank_2keep:
            df_lncol.append('Other')
            
        elif type(specific) == list:
            if crank in specific:
                df_lncol.append('Other')
                
        elif type(glom_d) == dict:
            if crank in glom_d.keys():
                df_lncol.append(glom_d[crank])
        else:
            df_lncol.append(crank)
    df[rank +"_n"] = df_lncol

# procesar taxonomia

In [ ]:
sintaxo = pd.read_csv('annotation/itsall_eukits.sintax.tsv', sep='\t', header=None, names=['sequence_name','full_annotation','strand','passed_annotation'])
sintaxo.info()

In [ ]:
# anotar de division a genero
for crank in ['d','p','c','o','f','g']:
    sintaxo[crank] = [np.nan if str(ci) == 'nan' or crank +':' not in ci
                      else ci.split(crank +':')[1].split(',')[0]
                      for ci in sintaxo['passed_annotation']]

In [ ]:
# anotar especie
sintaxo['s'] = [np.nan if str(crow['g']) == 'nan' or 's:' not in crow['passed_annotation']
                else crow['g'] +'_'+ crow['passed_annotation'].split('s:')[1]
                for _,crow in sintaxo.iterrows()]

In [ ]:
# extraer nombres de las secuencias
sintaxo['otu'] = [ci.split(';')[0]
                for ci in sintaxo['sequence_name']]

In [ ]:
sintaxo.loc[sintaxo['s'].notna()].shape

In [ ]:
sintaxo['d'].value_counts()

In [ ]:
tax = sintaxo.loc[sintaxo['d']=='Fungi', ['otu','d','p','c','o','f','g','s']]
tax.info()

In [ ]:
cranks = ['p','c','o','f','g','s']
for cranki,crank in enumerate(cranks):
    tax[crank] = [crow[crank] if str(crow[crank]) != 'nan'
                        else crow[cranks[cranki-1]]
                        for ci,crow in tax.iterrows()]

In [ ]:
tax

In [ ]:
tax.to_csv('itsall.tax.tsv',sep='\t', index=False)

In [ ]:
sintaxo

# tabla de conteos

In [ ]:
otur = pd.read_csv('clustering/itsall.97.tsv',sep='\t')
otur.info()

In [ ]:
# tamizar hongos
otu = otur.loc[otur['#OTU ID'].isin(list(tax['otu']))].copy()
otu.info()

In [ ]:
otu.index = otu['#OTU ID']
otu.drop(['#OTU ID'], axis=1, inplace=True)

In [ ]:
otu.to_csv('itsall.otu.csv')

In [ ]:
otu

# composicion taxonomica

In [ ]:
# crear una copia de la taxonomia
ctax = tax.copy()
# colapsar taxones de baja abundancia
glom_other_by_prop_rich_share(ctax, 'p', 0.05)
glom_other_by_prop_rich_share(ctax, 'c', 0.05)
glom_other_by_prop_rich_share(ctax, 'o', 0.05)
ctax = ctax.replace('Otros',' ')

In [ ]:
fig = px.sunburst(
    ctax,
    path = ['p_n', 'c_n', 'o_n'],
    width=800,
    height=800,
)
fig.show(renderer='colab')

In [ ]:
tax

# diversidad

In [ ]:
otu.columns

In [ ]:
# escribir abundancias
with open('itsall.abund.txt','w') as outfile:
    for csamp in otu.columns:
        outfile.write(csamp +','+ ','.join([ str(ci) for ci in list(otu.loc[otu[csamp]>0, csamp])]) +'\n')

In [ ]:
# cargar diversidades
divs = pd.read_csv('itsall.AsyEst.tsv', sep='\t')
divs.info()

In [ ]:
divs['Diversity'].unique()

In [ ]:
cdf = divs.loc[divs['Diversity']=='Species richness']
cdf.plot.bar('Assemblage', 'Observed')

In [ ]:
cdf = divs.loc[divs['Diversity']=='Shannon diversity']
cdf.plot.bar('Assemblage', 'Observed')

In [ ]:
cdf = divs.loc[divs['Diversity']=='Simpson diversity']
cdf.plot.bar('Assemblage', 'Observed')

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(5,8), sharex=True)

# agregar riqueza
cax=ax[0]
cvar='Species richness'
cdf = divs.loc[divs['Diversity']==cvar]
cdf.plot.bar('Assemblage', 'Observed', ax=cax, legend=False, color='grey')
cax.set_ylabel(cvar)

# agregar Hill-Shannnon
cax=ax[1]
cvar='Shannon diversity'
cdf = divs.loc[divs['Diversity']==cvar]
cdf.plot.bar('Assemblage', 'Observed', ax=cax, legend=False, color='grey')
cax.set_ylabel(cvar)

# agregar Hill-Simpson
cax=ax[2]
cvar='Simpson diversity'
cdf = divs.loc[divs['Diversity']==cvar]
cdf.plot.bar('Assemblage', 'Observed', ax=cax, legend=False, color='grey')
cax.set_ylabel(cvar)


# estructura

In [ ]:
# crear tabla derretida
cotu = otu.copy()
cotu['otu'] = cotu.index
cotu_melt = cotu.melt(id_vars=['otu'], value_vars=cotu.columns)
cotu_melt['site'] = cotu_melt['variable']
cotu_melt['count'] = cotu_melt['value']
cotu_melt = cotu_melt[['site', 'otu', 'count']]

In [ ]:
# agregar taxonomia
cranks = ['p','c','o','f','g','s']
for cranki,crank in enumerate(cranks):
    cotu_melt[crank] = [tax.loc[tax['otu']==crow['otu'],crank].item() for ci,crow in cotu_melt.iterrows()]

In [ ]:
cotu_melt.info()

## filo

In [ ]:
crank = 'p'

In [ ]:
# agrupar taxones con frecuencia reativa debajo del valor del corte
glom_other_by_prop(cotu_melt, crank, 0.001)
# revisar numero de taxones fuera de 'Otros'
cdf = pd.pivot_table(cotu_melt, values='count', index=[crank +'_n'],aggfunc="sum").sort_values('count')
print(cdf.shape)
cdf

In [ ]:
# crear tabla de proporciones y revisar que sume 1
cotu_ctax_counts = pd.pivot_table(cotu_melt, values='count', index=['site'], columns=[crank +'_n'],aggfunc="sum")
cotu_ctax_prop = cotu_ctax_counts.div(cotu_ctax_counts.sum(axis=1), axis=0)
cotu_ctax_prop.sum(axis=1)

In [ ]:
# revisar taxones mas abundantes
pd.pivot_table(cotu_melt, values='count', index=['site'],
                           columns=[crank],
                   aggfunc="sum").sum().sort_values(ascending=False).head(10)

In [ ]:
# generar agrupamiento jerarquico para el dendrograma
cotu_ctax_pivot = pd.pivot_table(cotu_melt, values='count', index=['site'],
                           columns=[crank],
                   aggfunc="sum")

# obtener matriz de disimilitud Bray-Curtis 
cotu_bc = beta_diversity("braycurtis", cotu_ctax_pivot.values, cotu_ctax_pivot.index)

# agrupamiento jerarquico
cmarkZ = linkage(cotu_bc.condensed_form(), method='complete')

# crear un dendrograma y obtener el orden de las filas
cotu_dendro = dendrogram(cmarkZ, labels=cotu_ctax_pivot.index, orientation='left', no_plot=True)
cotu_ordered_samples = cotu_dendro['ivl']

In [ ]:
# graficar dendrograma junto con estructura
fig, ax = plt.subplots(1, 2, figsize=(6, 6), gridspec_kw={'width_ratios': [2, 3]}, constrained_layout=True)

def no_color(x):
    return "#000000"  # Black color
# Plot the dendrogram
dendro = dendrogram(cmarkZ, labels=cotu_ctax_pivot.index, orientation='left', ax=ax[0])

# graficar estructura
cdf = cotu_melt
species_percent = cotu_ctax_prop
ordered_samples = cotu_ordered_samples

# 'tab20' colormap
tab20 = plt.get_cmap('tab20')

# crear diccionario de colores
unique_species_n = cdf[crank +'_n'].unique()
color_dict = {corder: tab20(i % 20) for i, corder in enumerate(unique_species_n)}
color_dict['Other'] = 'whitesmoke' # forzar color para 'Other'

# graficar barras apiladas con colores del diccionario
species_percent.loc[ordered_samples].plot.barh(stacked=True, color=[color_dict[col] for col in species_percent.columns],
                                               ax=ax[1], width=0.8)


ax[1].set_xlabel('')
ax[1].get_yaxis().set_visible(True)

plt.title(crank)
plt.legend(loc=(1.1,0.0))
# plt.savefig(f'{crank}.stackedbars.svg')
plt.show()

## diversidad beta

In [ ]:
nmds = MDS(n_components=2, metric=False, dissimilarity='precomputed')
cotu_bc_nmdsemb = nmds.fit_transform(cotu_bc.to_data_frame())

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

# create df
df_pcoa_tax = pd.DataFrame(cotu_bc_nmdsemb, columns=['NMDS1','NMDS2'], index=cotu_bc.to_data_frame().index)

# plot
df_pcoa_tax.fillna(0).plot('NMDS1', 'NMDS2', kind='scatter', ax=ax, s=20)

# anotar puntos
for ci, crow in df_pcoa_tax.iterrows():
    ax.annotate(ci, (crow['NMDS1']-0.01, crow['NMDS2']+0.01), size=6, color='grey')

plt.title(crank)
# plt.savefig(f'{crank}.nmds.svg')
plt.show()

In [ ]:
cotu_melt

# diversidad funcional

In [ ]:
funguild = pd.read_csv('/home/daemsel/fungaltraits-genus-species-guild_fg.tsv', sep='\t')
funguild.info()

In [ ]:
fdiv = tax.copy()
fdiv['gremio'] = [funguild.loc[funguild['species']==crow['s'], 'guild_fg'].head(1).item() if crow['s'] in list(funguild['species'])
                  else funguild.loc[funguild['Genus']==crow['g'], 'guild_fg'].head(1).item() if crow['g'] in list(funguild['Genus'])
                  else 'Desconocido'
                  for ci,crow in fdiv.iterrows()]
fdiv

In [ ]:
fdiv['gremio'].value_counts()

In [ ]:
# crear tabla derretida
cotu = otu.copy()
cotu['otu'] = cotu.index
cotu_melt = cotu.melt(id_vars=['otu'], value_vars=cotu.columns)
cotu_melt['site'] = cotu_melt['variable']
cotu_melt['count'] = cotu_melt['value']
cotu_melt = cotu_melt[['site', 'otu', 'count']]

In [ ]:
# agregar gremio
cotu_melt['gremio'] = [fdiv.loc[fdiv['otu']==crow['otu'],'gremio'].item() for ci,crow in cotu_melt.iterrows()]

In [ ]:
cotu_melt.info()

## gremio

In [ ]:
crank = 'gremio'

In [ ]:
# agrupar taxones con frecuencia reativa debajo del valor del corte
glom_other_by_prop(cotu_melt, crank, 0.001)
# revisar numero de taxones fuera de 'Otros'
cdf = pd.pivot_table(cotu_melt, values='count', index=[crank +'_n'],aggfunc="sum").sort_values('count')
print(cdf.shape)
cdf

In [ ]:
# crear tabla de proporciones y revisar que sume 1
cotu_ctax_counts = pd.pivot_table(cotu_melt, values='count', index=['site'], columns=[crank +'_n'],aggfunc="sum")
cotu_ctax_prop = cotu_ctax_counts.div(cotu_ctax_counts.sum(axis=1), axis=0)
cotu_ctax_prop.sum(axis=1)

In [ ]:
# revisar funciones mas abundantes
pd.pivot_table(cotu_melt, values='count', index=['site'],
                           columns=[crank],
                   aggfunc="sum").sum().sort_values(ascending=False).head(10)

In [ ]:
# generar agrupamiento jerarquico para el dendrograma
cotu_ctax_pivot = pd.pivot_table(cotu_melt, values='count', index=['site'],
                           columns=[crank],
                   aggfunc="sum")

# obtener matriz de disimilitud Bray-Curtis 
cotu_bc = beta_diversity("braycurtis", cotu_ctax_pivot.values, cotu_ctax_pivot.index)

# agrupamiento jerarquico
cmarkZ = linkage(cotu_bc.condensed_form(), method='complete')

# crear un dendrograma y obtener el orden de las filas
cotu_dendro = dendrogram(cmarkZ, labels=cotu_ctax_pivot.index, orientation='left', no_plot=True)
cotu_ordered_samples = cotu_dendro['ivl']

In [ ]:
# graficar dendrograma junto con estructura
fig, ax = plt.subplots(1, 2, figsize=(10, 6), gridspec_kw={'width_ratios': [2, 3]}, constrained_layout=True)

def no_color(x):
    return "#000000"  # Black color
# Plot the dendrogram
dendro = dendrogram(cmarkZ, labels=cotu_ctax_pivot.index, orientation='left', ax=ax[0])

# graficar estructura
cdf = cotu_melt
species_percent = cotu_ctax_prop
ordered_samples = cotu_ordered_samples

# 'tab20' colormap
tab20 = plt.get_cmap('tab20')

# crear diccionario de colores
unique_species_n = cdf[crank +'_n'].unique()
color_dict = {corder: tab20(i % 20) for i, corder in enumerate(unique_species_n)}
color_dict['Other'] = 'whitesmoke' # forzar color para 'Other'

# graficar barras apiladas con colores del diccionario
species_percent.loc[ordered_samples].plot.barh(stacked=True, color=[color_dict[col] for col in species_percent.columns],
                                               ax=ax[1], width=0.8)


ax[1].set_xlabel('')
ax[1].get_yaxis().set_visible(True)

plt.title(crank)
plt.legend(loc=(1.1,0.0))
# plt.savefig(f'{crank}.stackedbars.svg')
plt.show()

## diversidad beta

In [ ]:
nmds = MDS(n_components=2, metric=False, dissimilarity='precomputed')
cotu_bc_nmdsemb = nmds.fit_transform(cotu_bc.to_data_frame())

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

# create df
df_pcoa_fun = pd.DataFrame(cotu_bc_nmdsemb, columns=['NMDS1','NMDS2'], index=cotu_bc.to_data_frame().index)

# plot
df_pcoa_fun.fillna(0).plot('NMDS1', 'NMDS2', kind='scatter', ax=ax, s=20)

# anotar puntos
for ci, crow in df_pcoa_fun.iterrows():
    ax.annotate(ci, (crow['NMDS1']-0.01, crow['NMDS2']+0.01), size=6, color='grey')

plt.title(crank)
# plt.savefig(f'{crank}.nmds.svg')
plt.show()

In [ ]:
cotu_melt

# metadatos

In [ ]:
csamplemd = pd.read_csv('/home/daemsel/its.samplemd.tsv', sep='\t')
csamplemd

In [ ]:
# agregar diversidad
csamplemd['riqueza'] = [divs.loc[(divs['Assemblage']==crow['Sample'])&
    (divs['Diversity']=='Species richness'), 'Observed'].item() for _,crow in csamplemd.iterrows()]
csamplemd['shannon'] = [divs.loc[(divs['Assemblage']==crow['Sample'])&
    (divs['Diversity']=='Shannon diversity'), 'Observed'].item() for _,crow in csamplemd.iterrows()]
csamplemd['simpson'] = [divs.loc[(divs['Assemblage']==crow['Sample'])&
    (divs['Diversity']=='Simpson diversity'), 'Observed'].item() for _,crow in csamplemd.iterrows()]

In [ ]:
csamplemd

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(10,4), sharex=True, tight_layout=True)

# agregar riqueza
cax=ax[0]
cvar='riqueza'
sns.boxplot(x='Sistema', y=cvar, data=csamplemd, ax=cax)
cax.set_ylabel(cvar)

# agregar Hill-Shannnon
cax=ax[1]
cvar='shannon'
sns.boxplot(x='Sistema', y=cvar, data=csamplemd, ax=cax)
cax.set_ylabel(cvar)

# agregar Hill-Simpson
cax=ax[2]
cvar='simpson'
sns.boxplot(x='Sistema', y=cvar, data=csamplemd, ax=cax)
cax.set_ylabel(cvar)


In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

crank = 'filo'
# create df
df_pcoa_tax = pd.DataFrame(cotu_bc_nmdsemb, columns=['NMDS1','NMDS2'], index=cotu_bc.to_data_frame().index)
df_pcoa_tax['sistema'] = [csamplemd.loc[csamplemd['Sample']==ci, 'Sistema'].item() for ci in df_pcoa_tax.index]
df_pcoa_tax['sistema_c'] = ['chocolate' if ci == 'agro'
                            else 'green' for ci in df_pcoa_tax['sistema']]

# plot
df_pcoa_tax.fillna(0).plot('NMDS1', 'NMDS2', kind='scatter', ax=ax, s=20, color=df_pcoa_tax['sistema_c'])

# anotar puntos
for ci, crow in df_pcoa_tax.iterrows():
    ax.annotate(ci, (crow['NMDS1']-0.01, crow['NMDS2']+0.01), size=6, color='grey')

plt.title(crank)
# plt.savefig(f'{crank}.nmds.svg')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

crank = 'gremio'

# create df
df_pcoa_fun = pd.DataFrame(cotu_bc_nmdsemb, columns=['NMDS1','NMDS2'], index=cotu_bc.to_data_frame().index)
df_pcoa_fun['sistema'] = [csamplemd.loc[csamplemd['Sample']==ci, 'Sistema'].item() for ci in df_pcoa_fun.index]
df_pcoa_fun['sistema_c'] = ['chocolate' if ci == 'agro'
                            else 'green' for ci in df_pcoa_fun['sistema']]

# plot
df_pcoa_fun.fillna(0).plot('NMDS1', 'NMDS2', kind='scatter', ax=ax, s=20, color=df_pcoa_fun['sistema_c'])

# anotar puntos
for ci, crow in df_pcoa_fun.iterrows():
    ax.annotate(ci, (crow['NMDS1']-0.01, crow['NMDS2']+0.01), size=6, color='grey')

plt.title(crank)
# plt.savefig(f'{crank}.nmds.svg')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

# create df
df_pcoa_fun = pd.DataFrame(cotu_bc_nmdsemb, columns=['NMDS1','NMDS2'], index=cotu_bc.to_data_frame().index)
df_pcoa_fun['pH'] = [csamplemd.loc[csamplemd['Sample']==ci, 'pH'].item() for ci in df_pcoa_fun.index]

# plot
scatter = ax.scatter(df_pcoa_fun['NMDS1'], df_pcoa_fun['NMDS2'], s=20, c=df_pcoa_fun['pH'], cmap='plasma_r')

# anotar puntos
for ci, crow in df_pcoa_fun.iterrows():
    ax.annotate(ci, (crow['NMDS1']-0.01, crow['NMDS2']+0.01), size=6, color='grey')

# agregar barra de color
cbar = plt.colorbar(scatter, ax=ax,
                    shrink=0.3,
                    anchor=(0, 0.8),
                    pad=0.01)
cbar.set_label('pH', rotation=0, labelpad=20)

plt.title(crank)
# plt.savefig(f'{crank}.nmds.svg')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

# create df
df_pcoa_fun = pd.DataFrame(cotu_bc_nmdsemb, columns=['NMDS1','NMDS2'], index=cotu_bc.to_data_frame().index)
df_pcoa_fun['pH'] = [csamplemd.loc[csamplemd['Sample']==ci, 'pH'].item() for ci in df_pcoa_fun.index]
df_pcoa_fun['sistema'] = [csamplemd.loc[csamplemd['Sample']==ci, 'Sistema'].item() for ci in df_pcoa_fun.index]
df_pcoa_fun['sistema_c'] = ['chocolate' if ci == 'agro'
                            else 'green' for ci in df_pcoa_fun['sistema']]

# plot
scatter = ax.scatter(df_pcoa_fun['NMDS1'], df_pcoa_fun['NMDS2'], s=20,
                     c=df_pcoa_fun['pH'], cmap='plasma_r',
                     edgecolors=df_pcoa_fun['sistema_c'],  # Edge colors by vegetation
                     linewidths=1.5,)

# anotar puntos
for ci, crow in df_pcoa_fun.iterrows():
    ax.annotate(ci, (crow['NMDS1']-0.01, crow['NMDS2']+0.01), size=6, color='grey')

# agregar barra de color
cbar = plt.colorbar(scatter, ax=ax,
                    shrink=0.3,
                    anchor=(0, 0.8),
                    pad=0.01)
cbar.set_label('pH', rotation=0, labelpad=20)

# sistema (edge colors)
vegetation_handles = [
    plt.Line2D([0], [0], 
               marker='o', 
               color='w', 
               markeredgecolor=color,
               markerfacecolor=None,
               markeredgewidth=2,
               markersize=10, 
               label=csist)
    for csist, color in {'agro':'chocolate','forest':'green'}.items()
]

vegetation_legend = ax.legend(
    handles=vegetation_handles, 
    title='sistema',
    # loc='center left',
    bbox_to_anchor=(1, 0.45),
    frameon=True,
    fancybox=False,
    shadow=False
)

plt.title(crank)
# plt.savefig(f'{crank}.nmds.svg')
plt.show()